In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from progress.bar import Bar
from scraper_parts import get_links, check_proyecto_nuevo, scrape_proyecto_normal, scrape_proyecto_nuevo

pd.DataFrame({"A":[1,2]}).to_excel("prueba.xlsx", index=False)

links=get_links()
df_master = pd.DataFrame()
links_shortened = links[:5].copy()

print(f"Obteniendo {len(links)} casas")

with Bar("Processing --->", fill="##", suffix="%(percent).1f%% - %(eta)ds") as bar:
    for casa in links:
        is_proyecto_nuevo, soup, r, saltarse_link = check_proyecto_nuevo(casa)

        if is_proyecto_nuevo:
            scrape_proyecto_nuevo()
            continue
        else:        
            df_tmp = scrape_proyecto_normal(soup=soup, r=r)

        df_master = pd.concat([df_master, df_tmp])
        bar.next()

df_master.to_excel("resultado.xlsx", index=False)

Obteniendo 913 casas


In [2]:
casa

'https://www.encuentra24.com/el-salvador-es/bienes-raices-venta-de-propiedades-casas//24441764'

In [5]:
link="https://www.encuentra24.com/el-salvador-es/bienes-raices-proyectos-nuevos/comalapa-flats/24582258?list=category&catslug=bienes-raices-venta-de-propiedades-casas"
r = requests.get(link)
soup = BeautifulSoup(r.text, "html.parser")

In [15]:
soup.find(name="a", attrs={"class":"clase-loca-que-no-existe"})==None

True

In [5]:
not None

True